In [35]:
import pandas as pd
import json

# read file 
file_path = "./Mock_Data_Lumina.xlsx"  
df = pd.read_excel(file_path)

# Date -> datetime
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

# total_duration = suspended_duration_min + preparing_duration_min
df['total_duration'] = df['suspended_duration_min'] + df['preparing_duration_min']

# === group by models  ===
# 1. group by models and date, average duraion each day 
grouped_data_model = df.groupby(['Date', 'chargePointModel']).agg({
    'total_duration': 'mean'
}).reset_index()

chart_data_model = {
    'labels': grouped_data_model['Date'].dt.strftime('%Y-%m-%d').unique().tolist(),  
    'datasets': []
}

# create datasets
models = grouped_data_model['chargePointModel'].unique()
for model in models:
    model_data = grouped_data_model[grouped_data_model['chargePointModel'] == model]
    dataset = {
        'label': model,
        'data': model_data['total_duration'].tolist(),
        'borderColor': 'rgba(54, 162, 235, 1)',  
        'fill': False
    }
    chart_data_model['datasets'].append(dataset)

# 2. bar chart group by model - average duration 
grouped_data_model_bar = df.groupby('chargePointModel').agg({
    'suspended_duration_min': 'mean',
    'preparing_duration_min': 'mean'
}).reset_index()

grouped_data_model_bar['total_duration'] = grouped_data_model_bar['suspended_duration_min'] + grouped_data_model_bar['preparing_duration_min']
grouped_data_model_bar_sorted = grouped_data_model_bar.sort_values(by='total_duration', ascending=False)

chart_data_model_bar = {
    'labels': grouped_data_model_bar_sorted['chargePointModel'].tolist(),
    'datasets': [
        {
            'label': 'Suspended Duration',
            'data': grouped_data_model_bar_sorted['suspended_duration_min'].tolist(),
            'backgroundColor': 'rgba(54, 162, 235, 0.6)',  
            'stack': 'Stack 0'
        },
        {
            'label': 'Preparing Duration',
            'data': grouped_data_model_bar_sorted['preparing_duration_min'].tolist(),
            'backgroundColor': 'rgba(75, 192, 192, 0.6)', 
            'stack': 'Stack 0'
        }
    ]
}

# === group by main version ===
df['firmwareVersionGroup'] = df['firmwareVersion'].str.split('.').str[0]

# 3. group by date and version - average duration each day 
grouped_data_version = df.groupby(['Date', 'firmwareVersionGroup']).agg({
    'total_duration': 'mean'
}).reset_index()

chart_data_version = {
    'labels': grouped_data_version['Date'].dt.strftime('%Y-%m-%d').unique().tolist(), 
    'datasets': []
}

# create datasets
version_groups = grouped_data_version['firmwareVersionGroup'].unique()
for version_group in version_groups:
    version_data = grouped_data_version[grouped_data_version['firmwareVersionGroup'] == version_group]
    dataset = {
        'label': version_group,
        'data': version_data['total_duration'].tolist(),
        'borderColor': 'rgba(255, 99, 132, 1)', 
        'fill': False
    }
    chart_data_version['datasets'].append(dataset)

# 4. bar chart group by version - average duration 
grouped_data_version_bar = df.groupby('firmwareVersionGroup').agg({
    'suspended_duration_min': 'mean',
    'preparing_duration_min': 'mean'
}).reset_index()

grouped_data_version_bar['total_duration'] = grouped_data_version_bar['suspended_duration_min'] + grouped_data_version_bar['preparing_duration_min']
grouped_data_version_bar_sorted = grouped_data_version_bar.sort_values(by='total_duration', ascending=False)

chart_data_version_bar = {
    'labels': grouped_data_version_bar_sorted['firmwareVersionGroup'].tolist(),
    'datasets': [
        {
            'label': 'Suspended Duration',
            'data': grouped_data_version_bar_sorted['suspended_duration_min'].tolist(),
            'backgroundColor': 'rgba(54, 162, 235, 0.6)', 
            'stack': 'Stack 0'
        },
        {
            'label': 'Preparing Duration',
            'data': grouped_data_version_bar_sorted['preparing_duration_min'].tolist(),
            'backgroundColor': 'rgba(75, 192, 192, 0.6)', 
            'stack': 'Stack 0'
        }
    ]
}


# write into json file 
combined_chart_data = {
    'chart_model': chart_data_model,   
    'chart_version': chart_data_version  
}

# 导出为 multi_chart_data.json 文件
with open('multi_chart_data.json', 'w') as f:
    json.dump(combined_chart_data, f)

print("Succeed writing into multi_chart_data.json ")


Succeed writing into multi_chart_data.json 


In [36]:
# Group by Model: response_status_rejected + consumption_zero
df_grouped_model = df.groupby('chargePointModel').agg({
    'response_status_rejected': 'sum',
    'consumption_zero': 'sum'
}).reset_index()

df['firmwareVersionGroup'] = df['firmwareVersion'].str.split('.').str[0]
df_grouped_version = df.groupby('firmwareVersionGroup').agg({
    'response_status_rejected': 'sum',
    'consumption_zero': 'sum'
}).reset_index()

# Data grouped by model
chart_data_model_combine = {
    "labels": df_grouped_model['chargePointModel'].tolist(),
    "datasets": [
        {  # Bar chart: response_status_rejected
            "type": "bar",
            "label": "Response Status Rejected", 
            "data": df_grouped_model['response_status_rejected'].tolist(),
            "backgroundColor": "lightblue"
        },
        { # Line chart: consumption_zero
            "type": "line",
            "label": "Consumption Zero",  
            "data": df_grouped_model['consumption_zero'].tolist(),
            "borderColor": "red",
            "backgroundColor": "rgba(255, 99, 132, 0.2)",
            "fill": False
        }
    ]
}

# Data grouped by version
chart_data_version_combine = {
    "labels": df_grouped_version['firmwareVersionGroup'].tolist(),
    "datasets": [
        {
            "type": "bar",
            "label": "Response Status Rejected",  
            "data": df_grouped_version['response_status_rejected'].tolist(),
            "backgroundColor": "lightgreen"
        },
        {
            "type": "line",
            "label": "Consumption Zero", 
            "data": df_grouped_version['consumption_zero'].tolist(),
            "borderColor": "red",
            "backgroundColor": "rgba(255, 99, 132, 0.2)",
            "fill": False
        }
    ]
}


In [37]:

multi_chart_data = {
    'model_chart_line': chart_data_model,
    'model_chart_bar': chart_data_model_bar,
    'version_chart_line': chart_data_version,
    'version_chart_bar': chart_data_version_bar,
    'model_chart': chart_data_model_combine,
    'version_chart': chart_data_version_combine
}

# write into json file 
with open('multi_chart_data.json', 'w') as f:
    json.dump(multi_chart_data, f)



print("Succeed writing into multi_chart_data.json")

Succeed writing into multi_chart_data.json


In [38]:
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
issues_data = df.to_dict(orient='records')

with open('multi_chart_data.json', 'r') as f:
    multi_chart_data = json.load(f)

# write into multi_chart_data.json 
multi_chart_data['chargers_to_check'] = issues_data
with open('multi_chart_data.json', 'w') as f:
    json.dump(multi_chart_data, f)

print("Date updated in multi_chart_data.json")


Date updated in multi_chart_data.json
